# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import config
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from tqdm import tqdm
from GMHI import GMHI

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "metadata.csv"), index_col=[0, 1]).iloc[:, [0]]
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283294        0.000000                     0.000000   
         SAMN03283288        0.011865                     0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283294                                            0.000000   
         SAMN03283288                                            0.011865   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283294                                               0.000000                      
         SAMN03283288                                               0.011865                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283294                                               0.000000                                             
         SAMN03283288                                               0.011865                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283294                                               0.000000                                                                   
         SAMN03283288                                               0.011865                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283294                              True
         SAMN03283288                              True

# Generate group splits

In [3]:
# indentify groups to partition dataset

sample_studies = np.array(X.index.get_level_values(0))
o = OrdinalEncoder()
groups = o.fit_transform(sample_studies.reshape((len(sample_studies), 1))).flatten()
logo = LeaveOneGroupOut()

# Generate predictions in leave-one-study-out cross validation!

## GMHI

In [4]:
# instantiate the gmhi model
gmhi = GMHI(use_shannon=True, theta_f=1.4, theta_d=0.1)

# use only non-viral species
X_GMHI = X.copy()
X_GMHI = X_GMHI[list(filter(lambda x : "s__" in x and "virus" not in x, X_GMHI.columns))]

# generate predictions for each left out study (doing this manually because GMHI class required pandas dataframes)
GMHI_scores_ISV = np.zeros(y.shape)
for test_study in tqdm(np.unique(X.index.get_level_values(0))):
    train_studies = ~X_GMHI.index.get_level_values(0).isin([test_study])
    X_train = X_GMHI.loc[train_studies]
    y_train = y.loc[train_studies]
    X_test = X_GMHI.loc[test_study]
    gmhi.fit(X_train, y_train)
    GMHI_scores_ISV[X.index.get_level_values(0) == test_study] = np.expand_dims(gmhi.decision_function(X_test), -1)
    
GMHI_scores_ISV = pd.DataFrame(GMHI_scores_ISV, index=y.index, columns=["GMHI_ISV"])

100%|███████████████████████████████████████████████████████████████| 54/54 [00:08<00:00,  6.61it/s]


In [5]:
print(balanced_accuracy_score(y, GMHI_scores_ISV > 0))
idx = (abs(GMHI_scores_ISV) > 0.1).values
print(balanced_accuracy_score(y[idx], GMHI_scores_ISV[idx] > 0))
idx = (abs(GMHI_scores_ISV) > 1).values
print(balanced_accuracy_score(y[idx], GMHI_scores_ISV[idx] > 0))

0.7081303780383249
0.713050353228957
0.7706603340660031


## GMHI2

In [6]:
from sklearn.preprocessing import LabelEncoder

batches = LabelEncoder().fit_transform(X.index.get_level_values(0))
batches

array([ 0,  0,  0, ..., 52, 52, 52])

In [7]:
# instantiate the logistic regression classifier (gmhi2)
gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")

# generate logit predictions for each left out study
GMHI2_scores_ISV = cross_val_predict(gmhi2, X > config.PRESENCE_CUTOFF, y.values.flatten(), method="decision_function",
    groups=groups, cv=logo, verbose=2, n_jobs=-1
)

GMHI2_scores_ISV = pd.DataFrame(GMHI2_scores_ISV, index=y.index, columns=["GMHI2_ISV"])
GMHI2_scores_ISV

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done  51 out of  54 | elapsed:   14.8s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:   15.1s finished


GMHI2_ISV
Study_ID  Sample Accession           
GMHI-10   SAMN03283239       1.822810
          SAMN03283266       0.076991
          SAMN03283281       1.067728
          SAMN03283294       1.748159
          SAMN03283288       1.614673
...                               ...
Yang_2020 SRR6456373        -1.117766
          SRR6456374        -1.057515
          SRR6456375        -0.133311
          SRR6456376         0.883329
          SRR6456377        -0.249739

[8069 rows x 1 columns]

In [8]:
print(accuracy_score(y, GMHI2_scores_ISV > 0))
idx = (abs(GMHI2_scores_ISV) > 0.1).values
print(accuracy_score(y[idx], GMHI2_scores_ISV[idx] > 0))
idx = (abs(GMHI2_scores_ISV) > 0.5).values
print(accuracy_score(y[idx], GMHI2_scores_ISV[idx] > 0))
idx = (abs(GMHI2_scores_ISV) > 1).values
print(accuracy_score(y[idx], GMHI2_scores_ISV[idx] > 0))

0.7578386417152063
0.7702843725541352
0.8184184835665473
0.8656199492034172


In [9]:
# conf matrix

print(((GMHI2_scores_ISV > 0).values & ~y.values).sum())
print(((GMHI2_scores_ISV < 0).values & ~y.values).sum())
print(((GMHI2_scores_ISV > 0).values & y.values).sum())
print(((GMHI2_scores_ISV < 0).values & y.values).sum())

613
1909
4206
1341


# Save cv scores

In [10]:
GMHI_scores_ISV.to_csv(os.path.join(config.LOG_DIR, "GMHI_scores_ISV.csv"))
GMHI2_scores_ISV.to_csv(os.path.join(config.LOG_DIR, "GMHI2_scores_ISV.csv"))

# Nested ISV

In [11]:
logo_inner = LeaveOneGroupOut()
logo_outer = LeaveOneGroupOut()

In [12]:
p_grid = {"C" : [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3]}

In [54]:
import warnings
warnings.simplefilter("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [ ]:
from sklearn.model_selection import GridSearchCV

best_param = {}

for i, (train_index, test_index) in enumerate(logo_outer.split(X, y, groups)):
    print(i)
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]
    left_out_study = X_test.index.get_level_values(0)[0]
    
    # get the nested groups
    sample_studies = np.array(X_train.index.get_level_values(0))
    o = OrdinalEncoder()
    groups_inner = o.fit_transform(sample_studies.reshape((len(sample_studies), 1))).flatten()
    
    # instantiate the logistic regression classifier (gmhi2) without C parameter
    gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", class_weight="balanced")
    
    # setup grid search
    gridsearch = GridSearchCV(gmhi2, p_grid, cv=logo_inner, verbose=0, n_jobs=-1, scoring="balanced_accuracy")
    
    # perform grid search and fit model using best hyperparameters (only using training fold)
    gridsearch.fit(X_train > config.PRESENCE_CUTOFF, y_train.values.flatten(), groups=groups_inner)
    
    # save grid search results
    print(gridsearch.best_params_["C"])
    best_param[left_out_study] = pd.DataFrame(gridsearch.cv_results_)

In [26]:
len(best_param)

54

In [39]:
performance_list = []

for study in best_param:
    study_results = best_param[study][["param_C", "mean_test_score"]]
    study_results = study_results.set_index("param_C").T
    study_results.index = [study]
    performance_list.append(study_results)

In [44]:
nested_ISV = pd.concat(performance_list)

In [68]:
nested_ISV.to_csv(os.path.join(config.LOG_DIR, "nested_ISV.csv"))

In [59]:
nested_ISV

param_C,0.001,0.003,0.010,0.030,0.100,0.300,1.000,3.000
GMHI-10,0.377878,0.688777,0.718172,0.734931,0.721041,0.707026,0.699737,0.686152
GMHI-11,0.359010,0.694704,0.717342,0.734440,0.714313,0.704529,0.698254,0.685371
GMHI-13,0.377878,0.686258,0.709212,0.726867,0.710154,0.699594,0.694469,0.681922
GMHI-14,0.377878,0.686786,0.715453,0.738386,0.733129,0.716430,0.706952,0.694526
GMHI-15,0.377878,0.665039,0.716664,0.729634,0.714350,0.704431,0.702464,0.679471
GMHI-17,0.366071,0.693061,0.725034,0.732437,0.721876,0.711686,0.702845,0.692503
GMHI-2,0.365338,0.676216,0.717464,0.730265,0.714184,0.703579,0.697088,0.685472
GMHI-23,0.365998,0.688237,0.715263,0.735327,0.723408,0.705780,0.704405,0.692258
GMHI-24,0.377878,0.688238,0.713889,0.728262,0.715533,0.705935,0.700427,0.682969
GMHI-26,0.377878,0.685027,0.713149,0.728978,0.712468,0.702430,0.696631,0.684436


In [66]:
all(nested_ISV.idxmax(axis=1) == 0.03)

True